In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
from ICARUS.database import DB
from ICARUS.database.database3D import Database_3D
from time import sleep

# Open Database

In [ ]:
db3d: Database_3D = DB.vehicles_db
db3d.load_data()

### Select Plane

In [ ]:
planenames: list[str] = db3d.get_planenames()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}") 
sleep(1)

indexes = [0]# range(0,len(planenames))
print(len(planenames))
plane = db3d.planes[planenames[indexes[0]]]
# dynplane = db3d.dyn_planes[planenames[index]]
planenames = [planenames[i] for i in indexes]

# Add XFLR Polars

In [ ]:
from ICARUS.computation.solvers.XFLR5.polars import read_polars_3d
from ICARUS.database import EXTERNAL_DB
for name in planenames:
    if name.startswith("XFLR"):
        continue

    if f"XFLR_{name}" not in planenames: 
        try:
            import os
            XFLR5PLANEDIR: str = os.path.join(EXTERNAL_DB,f"{name}.txt")
            read_polars_3d(XFLR5PLANEDIR, name)
            print(f"Imported XFLR polar for {name}")
            planenames.append(f"XFLR_{name}")
        except FileNotFoundError:
            print(f"No XFLR polar found for {name}")
            pass
        

# Airplane Visualization

In [ ]:
# plane.visualize()

# Polars

In [ ]:
from ICARUS.visualization.airplane.db_polars import plot_airplane_polars 
DB.vehicles_db.load_data()
solvers = [
            #  "GenuVP3 Potential",
            #  "GenuVP3 2D", 
            # "GNVP3 ONERA",
            # "GNVP7 Potential",
            # "GNVP7 2D",
            #  "LSPT Potential",
            # "LSPT 2D",/'/
            "AVL"
        ]
axs, fig = plot_airplane_polars(
    planenames,
    solvers,
    plots = [["AoA", "CL"], ["AoA", "CD"], ["AoA", "Cm"]],
    size=(6,7),
)

In [ ]:
from ICARUS.visualization.airplane.cg_investigation import cg_investigation

cg_investigation(
    planenames[0],
    solvers,
    size=(10,7),
)

# Case Convergence

In [ ]:
from ICARUS.visualization.airplane.gnvp_convergence import plot_convergence

In [ ]:
angles2show = [0.]# All
# Only Works for unsteady solvers: LSPT, GNVP3, GNVP7
plot_convergence(
    plane.name,
    angles = angles2show,
    solvers= ["All"],
    plot_error = False,
    size=(16,7)
)

# Wake Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_wake import plot_gnvp_wake
from ICARUS.database.utils  import angle_to_case

In [ ]:
CASE: str = angle_to_case(0.)
plane = db3d.planes[plane.name]
# plane = db3d.planes['bmark']
# plane.visualize()
plot_gnvp_wake(3,plane, CASE, scale = True)

 # Loads Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_strips import gnvp_strips_2d, gnvp_strips_3d
NBs = [i+1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ",NBs)

stripData = gnvp_strips_2d(plane,CASE, NBs[0], category= 'Wind')

# Dynamics

In [ ]:
states = db3d.states['bmark']
for i,name in enumerate(states):
    print(f"{i}: {name}")

In [ ]:
db3d.states['bmark']

In [ ]:
from ICARUS.flight_dynamics.state import State

state: State = states['plane_1']
state.stability_fd()
print(state)
state.plot_eigenvalues()

In [ ]:
state.state_space